In [4]:
#import shutil as shutil
#shutil.which("python3")


In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import re as re

#current_path = Path(__file__).parent
file_path = "/home/aurelien/Programation/telegramSA/Data/messages.html"


with open(file_path) as fp:
    soup = BeautifulSoup(fp, "lxml")

In [1]:
#soup.contents

In [2]:
#print(soup.find(attrs={"class" : "message default clearfix"}).prettify())

In [14]:
raw_messages = soup.find_all(attrs = {"class" : "message default clearfix"})

def findMedia(html_msg):
    ret = None
    if html_msg.find(attrs = {"class" : "media"}):
        ret = html_msg
    return ret

media_lst = [ findMedia(media) for media in raw_messages]

print(media_lst[0].prettify())

<div class="message default clearfix" id="message173603">
 <div class="pull_left userpic_wrap">
  <div class="userpic userpic8" style="width: 42px; height: 42px">
   <div class="initials" style="line-height: 42px">
    FP
   </div>
  </div>
 </div>
 <div class="body">
  <div class="pull_right date details" title="01.06.2019 13:46:08">
   13:46
  </div>
  <div class="from_name">
   Fiona Paul
  </div>
  <div class="media_wrap clearfix">
   <div class="media clearfix pull_left media_photo">
    <div class="fill pull_left">
    </div>
    <div class="body">
     <div class="title bold">
      Photo
     </div>
     <div class="description">
      Not included, change data exporting settings to download.
     </div>
     <div class="status details">
      960x1280, 609.9 KB
     </div>
    </div>
   </div>
  </div>
 </div>
</div>



In [7]:
raw_messages = soup.find_all(attrs = {"class" : "message default clearfix"})

def cleanHashtags(html_message_list):
    try:
        ret = [hashtag.string.strip("#") for hashtag in html_message_list]
    except:
        ret = html_message_list
    return ret

def cleanLinks(html_links_list):
    ret = [link["href"] for link in html_links_list]
    return ret

def extractMessage(html_message):
    msg = { "message_id" : html_message['id'],
            "time" : html_message.find(attrs = {"class" : "date"})['title'],
            "user" : html_message.find(attrs = {"class" : "from_name"}).string,
            "message" :  None,
            "hashtag" : cleanHashtags(html_message.find_all(attrs = {"onclick" : re.compile("ShowHashtag(.*)")})),
            "links" : cleanLinks(html_message.find_all(href=re.compile("."))),
            "raw_message" : html_message
    }
    if html_message.find(attrs = {"class" : "text"}):
        msg["message"] = html_message.find(attrs = {"class" : "text"}).next_element
        # using the fact that .next_element return the inside of the message which is empty before 
        # the next <a> tag inside (for cases of link/hashtags)
    else:
        msg["message"] = "not found"
    try:
        msg["user"] = msg["user"].strip()
        msg["message"] = msg["message"].strip()
    except:
        pass
    return msg

msg_list = [extractMessage(items) for items in raw_messages]
msg_list[0]

#for l in msg_list:
#    if l["message"] == "":
#        print("###########")
#        print(l["raw_message"])

{'message_id': 'message173603',
 'time': '01.06.2019 13:46:08',
 'user': 'Fiona Paul',
 'message': 'not found',
 'hashtag': [],
 'links': [],
 'raw_message': <div class="message default clearfix" id="message173603">
 <div class="pull_left userpic_wrap">
 <div class="userpic userpic8" style="width: 42px; height: 42px">
 <div class="initials" style="line-height: 42px">
 FP
         </div>
 </div>
 </div>
 <div class="body">
 <div class="pull_right date details" title="01.06.2019 13:46:08">
 13:46
        </div>
 <div class="from_name">
 Fiona Paul
        </div>
 <div class="media_wrap clearfix">
 <div class="media clearfix pull_left media_photo">
 <div class="fill pull_left">
 </div>
 <div class="body">
 <div class="title bold">
 Photo
           </div>
 <div class="description">
 Not included, change data exporting settings to download.
           </div>
 <div class="status details">
 960x1280, 609.9 KB
           </div>
 </div>
 </div>
 </div>
 </div>
 </div>}